In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark

In [3]:
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('Quijote')\
        .getOrCreate()

Capítulo 2
*  Descargar el Quijote 
https://gist.github.com/jsdario/6d6c69398cb0c73111e49f1218960f79
Aplicar no solo count (para obtener el número de líneas) y show sino probar 
distintas sobrecargas del método show (con/sin truncate, indicando/sin indicar 
num de filas, etc) así como también los métodos, head, take, first (diferencias 
entre estos 3?)

In [4]:
quijote = spark.read.text('el_quijote.txt')

In [6]:
quijote.count()

2186

In [7]:
quijote.show(5, truncate= False)

+----------------------------------------------------------------------------------------------+
|value                                                                                         |
+----------------------------------------------------------------------------------------------+
|DON QUIJOTE DE LA MANCHA                                                                      |
|Miguel de Cervantes Saavedra                                                                  |
|                                                                                              |
|PRIMERA PARTE                                                                                 |
|CAPÍTULO 1: Que trata de la condición y ejercicio del famoso hidalgo D. Quijote de la Mancha|
+----------------------------------------------------------------------------------------------+
only showing top 5 rows



In [9]:
quijote.show(truncate= False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
quijote.show(truncate= True)

+--------------------+
|               value|
+--------------------+
|DON QUIJOTE DE LA...|
|Miguel de Cervant...|
|                    |
|       PRIMERA PARTE|
|CAPÍTULO 1: Que ...|
|En un lugar de la...|
|Tuvo muchas veces...|
|En resolución, e...|
|historia más cie...|
|Decía él, que e...|
|En efecto, remata...|
|Imaginábase el p...|
|linaje y patria, ...|
|Limpias, pues, su...|
|Capítulo 2: Que ...|
|Hechas, pues, est...|
|Estos pensamiento...|
|Con estos iba ens...|
|Autores hay que d...|
|muertos de hambre...|
+--------------------+
only showing top 20 rows



In [14]:
quijote.head(5)

[Row(value='DON QUIJOTE DE LA MANCHA'),
 Row(value='Miguel de Cervantes Saavedra'),
 Row(value=''),
 Row(value='PRIMERA PARTE'),
 Row(value='CAPÍTULO 1: Que trata de la condición y ejercicio del famoso hidalgo D. Quijote de la Mancha')]

In [13]:
quijote.take(5)

[Row(value='DON QUIJOTE DE LA MANCHA'),
 Row(value='Miguel de Cervantes Saavedra'),
 Row(value=''),
 Row(value='PRIMERA PARTE'),
 Row(value='CAPÍTULO 1: Que trata de la condición y ejercicio del famoso hidalgo D. Quijote de la Mancha')]

In [17]:
quijote.first()

Row(value='DON QUIJOTE DE LA MANCHA')

In [18]:
# First()------> Devuelve el primer elemento
# take()-------> Funciona escaneando una partición
# head()-------> Devuelve las primeras n filas

* Del ejercicio de M&M aplicar: 
> Otras operaciones de agregación como el Max con otro tipo de 
ordenamiento (descendiente).

> hacer un ejercicio como el “where” de CA que aparece en el libro pero 
indicando más opciones de estados (p.e. NV, TX, CA, CO).

> Hacer un ejercicio donde se calculen en una misma operación el Max, 
Min, Avg, Count. Revisar el API (documentación) donde encontrarán 
este ejemplo:
ds.agg(max($"age"), avg($"salary"))
ds.groupBy().agg(max($"age"), avg($"salary"))
NOTA: $ es un alias de col()

> Hacer también ejercicios en SQL creando tmpVie

In [19]:
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('MnM')\
        .getOrCreate()

In [34]:
mnm_df = (spark.read.format('csv')
         .option('header', 'true')
         .option('inferSchema', 'true')
         .load('data/mnm_dataset.csv'))

In [35]:
mnm.schema

StructType(List(StructField(State,StringType,true),StructField(Color,StringType,true),StructField(Count,IntegerType,true)))

In [36]:
mnm.show()

+-----+------+-----+
|State| Color|Count|
+-----+------+-----+
|   TX|   Red|   20|
|   NV|  Blue|   66|
|   CO|  Blue|   79|
|   OR|  Blue|   71|
|   WA|Yellow|   93|
|   WY|  Blue|   16|
|   CA|Yellow|   53|
|   WA| Green|   60|
|   OR| Green|   71|
|   TX| Green|   68|
|   NV| Green|   59|
|   AZ| Brown|   95|
|   WA|Yellow|   20|
|   AZ|  Blue|   75|
|   OR| Brown|   72|
|   NV|   Red|   98|
|   WY|Orange|   45|
|   CO|  Blue|   52|
|   TX| Brown|   94|
|   CO|   Red|   82|
+-----+------+-----+
only showing top 20 rows



In [51]:
from pyspark.sql.functions import count, max, min, avg
count_mnm_df = (mnm
               .select('State', 'Color', 'Count')
               .groupBy('State', 'Color')
               .agg(max('Count').alias('Max'),
                   min('Count').alias('Min'))
               .orderBy('Max', ascending = False))

In [45]:
count_mnm_df.show()

+-----+------+---+---+
|State| Color|Max|Min|
+-----+------+---+---+
|   WY| Green|100| 10|
|   NV|   Red|100| 10|
|   UT|  Blue|100| 10|
|   WA|Orange|100| 10|
|   NM| Green|100| 10|
|   CA|  Blue|100| 10|
|   WA|   Red|100| 10|
|   NV| Brown|100| 10|
|   AZ| Green|100| 10|
|   CA|   Red|100| 10|
|   AZ|Orange|100| 10|
|   CO|  Blue|100| 10|
|   NM|Orange|100| 10|
|   NM|Yellow|100| 10|
|   WY|Orange|100| 10|
|   UT|Yellow|100| 10|
|   WY|   Red|100| 10|
|   OR|  Blue|100| 10|
|   NV|Orange|100| 10|
|   AZ|Yellow|100| 10|
+-----+------+---+---+
only showing top 20 rows



In [48]:
wy_count_mnm_df = (mnm_df
                  .select('State', 'Color', 'Count')
                  .where(mnm_df.State == 'WY')
                  .groupBy('State', 'Color')
                  .agg(count('Count').alias('Total'))
                  .orderBy('Total', ascending= False))
wy_count_mnm_df.show()

+-----+------+-----+
|State| Color|Total|
+-----+------+-----+
|   WY| Green| 1695|
|   WY|   Red| 1670|
|   WY|  Blue| 1664|
|   WY|Yellow| 1626|
|   WY|Orange| 1595|
|   WY| Brown| 1532|
+-----+------+-----+



In [49]:
nv_count_mnm_df = (mnm_df
                  .select('State', 'Color', 'Count')
                  .where(mnm_df.State == 'NV')
                  .groupBy('State', 'Color')
                  .agg(count('Count').alias('Total'))
                  .orderBy('Total', ascending= False))
nv_count_mnm_df.show()

+-----+------+-----+
|State| Color|Total|
+-----+------+-----+
|   NV|Orange| 1712|
|   NV| Green| 1698|
|   NV|Yellow| 1675|
|   NV|  Blue| 1673|
|   NV| Brown| 1657|
|   NV|   Red| 1610|
+-----+------+-----+



In [50]:
tx_count_mnm_df = (mnm_df
                  .select('State', 'Color', 'Count')
                  .where(mnm_df.State == 'TX')
                  .groupBy('State', 'Color')
                  .agg(count('Count').alias('Total'))
                  .orderBy('Total', ascending= False))
tx_count_mnm_df.show()

+-----+------+-----+
|State| Color|Total|
+-----+------+-----+
|   TX| Green| 1737|
|   TX|   Red| 1725|
|   TX|Yellow| 1703|
|   TX|Orange| 1652|
|   TX| Brown| 1641|
|   TX|  Blue| 1614|
+-----+------+-----+



In [52]:
mnm_df = (mnm
        .select('State', 'Color', 'Count')
        .groupBy('State', 'Color')
        .agg(max('Count').alias('Max'),
             min('Count').alias('Min'),
            avg('Count').alias('Avg'))
               .orderBy('Max', ascending = False))
mnm_df.show()

+-----+------+---+---+------------------+
|State| Color|Max|Min|               Avg|
+-----+------+---+---+------------------+
|   WY| Green|100| 10|55.657227138643066|
|   NV|   Red|100| 10|  55.4944099378882|
|   UT|  Blue|100| 10|54.366767371601206|
|   WA|Orange|100| 10|55.199638118214715|
|   NM| Green|100| 10|  54.1973840665874|
|   CA|  Blue|100| 10| 55.59762944479102|
|   WA|   Red|100| 10| 55.85397965290245|
|   NV| Brown|100| 10| 55.81050090525045|
|   AZ| Green|100| 10| 54.82219570405728|
|   CA|   Red|100| 10| 55.26992753623188|
|   AZ|Orange|100| 10| 54.28300769686205|
|   CO|  Blue|100| 10| 55.11032448377581|
|   NM|Orange|100| 10|  54.8054054054054|
|   NM|Yellow|100| 10| 54.94490521327014|
|   WY|Orange|100| 10|55.144827586206894|
|   UT|Yellow|100| 10|54.263829787234044|
|   WY|   Red|100| 10|54.950898203592814|
|   OR|  Blue|100| 10| 54.99756986634265|
|   NV|Orange|100| 10|54.865070093457945|
|   AZ|Yellow|100| 10| 54.98548972188634|
+-----+------+---+---+------------

In [58]:
# In SQL

mnm.createOrReplaceTempView('temp_table')

spark.sql('SELECT * FROM temp_table').show()

+-----+------+-----+
|State| Color|Count|
+-----+------+-----+
|   TX|   Red|   20|
|   NV|  Blue|   66|
|   CO|  Blue|   79|
|   OR|  Blue|   71|
|   WA|Yellow|   93|
|   WY|  Blue|   16|
|   CA|Yellow|   53|
|   WA| Green|   60|
|   OR| Green|   71|
|   TX| Green|   68|
|   NV| Green|   59|
|   AZ| Brown|   95|
|   WA|Yellow|   20|
|   AZ|  Blue|   75|
|   OR| Brown|   72|
|   NV|   Red|   98|
|   WY|Orange|   45|
|   CO|  Blue|   52|
|   TX| Brown|   94|
|   CO|   Red|   82|
+-----+------+-----+
only showing top 20 rows



In [71]:
mnm.createOrReplaceTempView('temp_table')
spark.sql('''SELECT State, Color,  MAX(Count) Maximo,
            MIN(Count) Minimo, AVG(Count) Media FROM temp_table
            GROUP BY State, Color
            ''').show()

+-----+------+------+------+------------------+
|State| Color|Maximo|Minimo|             Media|
+-----+------+------+------+------------------+
|   WY| Green|   100|    10|55.657227138643066|
|   NV|   Red|   100|    10|  55.4944099378882|
|   UT|  Blue|   100|    10|54.366767371601206|
|   WA|Orange|   100|    10|55.199638118214715|
|   NM| Green|   100|    10|  54.1973840665874|
|   CA|  Blue|   100|    10| 55.59762944479102|
|   WA|   Red|   100|    10| 55.85397965290245|
|   NV| Brown|   100|    10| 55.81050090525045|
|   AZ| Green|   100|    10| 54.82219570405728|
|   CA|   Red|   100|    10| 55.26992753623188|
|   AZ|Orange|   100|    10| 54.28300769686205|
|   CO|  Blue|   100|    10| 55.11032448377581|
|   NM|Orange|   100|    10|  54.8054054054054|
|   NM|Yellow|   100|    10| 54.94490521327014|
|   WY|Orange|   100|    10|55.144827586206894|
|   UT|Yellow|   100|    10|54.263829787234044|
|   WY|   Red|   100|    10|54.950898203592814|
|   OR|  Blue|   100|    10| 54.99756986